In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [5]:
%run ./layers.ipynb

## Graph Convolution Network module
The GCN module represents our graph network capable of processing our ingredients for each recipe.
output: ingredient features

In [10]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, dropout, init):
        super(GCN, self).__init__()
        
        _, vec = torchwordemb.load_word2vec_bin('../data/vocab.bin')
        self.embs = nn.Embedding(vec.size(0), 300, padding_idx=0)
        self.gc1 = GraphConvolution(nfeat, nhid, init=init)
        self.gc2 = GraphConvolution(nhid, nhid, init=init)
        self.dropout = dropout
    
    def forward(self, x, adj):
        x = self.embs(x) 
        x = F.dropout(F.relu(self.gc1(x, adj)), self.dropout, training=self.training)
        x = self.gc2(x, adj)

        return F.log_softmax(x, dim=1)

##  LSTM module 
Molule for processing instructions
output: instruction features

In [ ]:

class LSTM(nn.Module):
    def __init__(self, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hiddMoen_size
        self.num_layers = 1
        self.input_size = hidden_size
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers)

    def forward(self, embedded_input, hx, cx):
        output, (hx, cx) = self.lstm(embedded_input, (hx, cx))
        return output, hx, cx

    def initHidden(self):

        hx = torch.zeros(self.num_layers, 1, self.hidden_size)
        cx = torch.zeros(self.num_layers, 1, self.hidden_size)
        return hx, cx



## Resnet Module
Module for processing images.
output: image features

In [ ]:
class RESNET(nn.Module):
    def __init__(self):
        super(RESNET, self).__init__()
        resnet = models.resnet50(pretrained=True)
        modules = list(resnet.children())[:-1]  # we do not use the last fc layer.
        self.visionMLP = nn.Sequential(*modules)
        self.visual_embedding = nn.Sequential(nn.Linear(2048, 1024),nn.Tanh())

    def forward(self, x):
        visual_emb = self.visionMLP(x)
        visual_emb = visual_emb.view(visual_emb.size(0), -1)
        visual_emb = self.visual_embedding(visual_emb)
        visual_emb = norm(visual_emb)
        return visual_emb

def norm(input, p=2, dim=1, eps=1e-12):
    return input / input.norm(p,dim,keepdim=True).clamp(min=eps).expand_as(input)

## Joint Module
Module for learning recipe embeddings which concatenate ingredient vector and instruction vector .
output: Recipe embedding

In [ ]:
class model(nn.Module):
    def __init__(self,lstm, gcn ):
        super(Joint_model, self).__init__()
        self.gcn=gcn
        self.lstm=lstm
        self.linear = nn.Linear(2048, 1024)


    def forward(self, X):
        inst_embed=self.lstm(X["instrs"])
        ing_embed=self.gcn(X["ingrs"])

        recipe_embed = self.linear(torch.cat(inst_embed, ing_embed))
        return  recipe_embed